In [10]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import re
import jieba
import jieba.analyse

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [11]:
def cutval(df):
    infocut= []
    for d in df["\n            《脫稿玩家》 故事描述一個銀行小職員 - 蓋伊 (萊恩雷諾斯) 日復一日年復一年做著無聊的工作，過著樸實無華的生活，直到有一天他突然發現自己是一個暴力電玩中可有可無的配角，他想要突破現狀，但這個動作竟讓整個遊戲大變調，除了要挑戰層層關卡，還要拯救世界，更重要的是否能從C咖一躍成A咖呢？"]:
        info= jieba.analyse.extract_tags(d, topK=2)
        infocut.append(info)

    typecut= []
    for d in df["[' 動作 ', ' 喜劇 ']"]:
        ch= re.compile("[\u4e00-\u9fa5]")
        seg_word=  "".join(ch.findall(d))
        info= jieba.lcut(seg_word)
        typecut.append(info)

    namecut= []
    for d in df['脫稿玩家']:
        ch= re.compile("[\u4e00-\u9fa5]")
        name=  "".join(ch.findall(d))
        namecut.append(name)

    data= { 'type':typecut, 'name':namecut, 'info':infocut}
    df1= pd.DataFrame(data)

    df1['type']= df1['type'].apply(lambda x :  str(x)[1:-2].replace("'", "").replace(",", ""))
    df1['info']= df1['info'].apply(lambda x :  str(x)[1:-2].replace("'", "").replace(",", ""))
    df1['key']= df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['info'].astype(str)
    return df1

In [12]:
def tfidf(df):
    vectorizer= CountVectorizer()
    X= vectorizer.fit_transform(df['key'])
    tfidf= TfidfTransformer() 
    tf= tfidf.fit_transform(X)
    word= vectorizer.get_feature_names()
    
    return tf

In [20]:
def knn(X,Y):
    s=500/5695
    X_train, X_test, y_train, y_test= train_test_split(X, Y['type'].str[0:2], test_size= s, random_state= 1)
    clf=KNeighborsClassifier(n_neighbors= 50, p=2, weights= 'distance')
    clf.fit(X_train, y_train)
    clf.predict(X_test)
    o=clf.score(X_test, y_test)
    print(o)

In [21]:
df = pd.read_csv('Info1.csv')
df1 = cutval(df) 
tf = tfidf(df1)
knn(tf,df1)

0.77
